# Efficient Yelp API Calls (Core)
- David Atkins

In [3]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [4]:
# Loading API Credentials
with open('/Users/gondr/.secret/yelp_api.json') as f:
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [25]:
# set our API call parameters 
LOCATION = 'Boston,MA'
TERM = 'Reuben'

In [26]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "data/results_in_progress_boston_reuben.json"
JSON_FILE

'data/results_in_progress_boston_reuben.json'

In [27]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [28]:
create_json_file(JSON_FILE, delete_if_exists=True)
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)

total_results = results['total']

results_per_page = len(results['businesses'])

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] data/results_in_progress_boston_reuben.json not found. Saving empty list to new file.
- 0 previous results found.


25

In [29]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/25 [00:00<?, ?it/s]

In [30]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,w2F5N4h26hOrz2KoxThomw,sam-lagrassas-boston-3,Sam LaGrassa's,https://s3-media4.fl.yelpcdn.com/bphoto/0Dx6vF...,False,https://www.yelp.com/biz/sam-lagrassas-boston-...,1173,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 42.35762673340388, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '44 Province St', 'address2': '',...",+16173576861,(617) 357-6861,1459.065431
1,EbUZhM4fLpsWQ8fpBhhgEQ,mike-and-pattys-boston-boston,Mike & Patty's - Boston,https://s3-media4.fl.yelpcdn.com/bphoto/CLxvYZ...,False,https://www.yelp.com/biz/mike-and-pattys-bosto...,2108,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 42.348561, 'longitude': -71.067928}","[pickup, delivery]",$$,"{'address1': '12 Church St', 'address2': '', '...",+16174233447,(617) 423-3447,283.948363
2,w-i1JcbJDuJmoZbGjxBH8w,charlies-sandwich-shoppe-boston,Charlie's Sandwich Shoppe,https://s3-media1.fl.yelpcdn.com/bphoto/mQoJq3...,False,https://www.yelp.com/biz/charlies-sandwich-sho...,342,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 42.3441143, 'longitude': -71.0777...","[pickup, delivery]",$$,"{'address1': '429 Columbus Ave', 'address2': '...",+16175367669,(617) 536-7669,675.941589
3,n6iTeADfaVagZjsoZZ631A,emmets-irish-pub-and-restaurant-boston,Emmet's Irish Pub & Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/lB5g76...,False,https://www.yelp.com/biz/emmets-irish-pub-and-...,432,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.0,"{'latitude': 42.3582161, 'longitude': -71.0612...",[delivery],$$,"{'address1': '6 Beacon St', 'address2': '', 'a...",+16177428565,(617) 742-8565,1414.937330
4,-WG2sv6ApNh1fa7ot-QAEA,archies-new-york-deli-boston,Archie's New York Deli,https://s3-media2.fl.yelpcdn.com/bphoto/qJM_tc...,False,https://www.yelp.com/biz/archies-new-york-deli...,218,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 42.3550721034927, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '101 Arch St', 'address2': '', 'a...",+16173305145,(617) 330-5145,1275.006803


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
480,O7-iODKfhUj667cgFNr3hw,tgi-fridays-braintree-3,TGI Fridays,https://s3-media3.fl.yelpcdn.com/bphoto/QvlHlS...,False,https://www.yelp.com/biz/tgi-fridays-braintree...,381,"[{'alias': 'tradamerican', 'title': 'American ...",2.0,"{'latitude': 42.2208707, 'longitude': -71.0297...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '60 Forbes Rd', 'address2': None,...",+17818483133,(781) 848-3133,14540.998389
481,uQ08w_MZPRfkK6bdBtfQFg,supreme-pizza-quincy,Supreme Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/g9cUN_...,False,https://www.yelp.com/biz/supreme-pizza-quincy?...,11,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"{'latitude': 42.2406616210938, 'longitude': -7...",[],$,"{'address1': '271 Quincy Ave', 'address2': '',...",+16177703888,(617) 770-3888,13688.049778
482,TGtRW7nq7i4fLAzrKdaaaQ,angelinas-pizzeria-braintree,Angelina's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/X9a_1F...,False,https://www.yelp.com/biz/angelinas-pizzeria-br...,134,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 42.2263336181641, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '419 Elm St', 'address2': '', 'ad...",+17818437827,(781) 843-7827,14977.655123
483,9F8-_by7AXe1d3uitgT6gA,ny-pie-waltham-2,NY Pie,https://s3-media3.fl.yelpcdn.com/bphoto/76FUSH...,False,https://www.yelp.com/biz/ny-pie-waltham-2?adju...,287,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 42.360123512927, 'longitude': -71...","[pickup, delivery]",$$,"{'address1': '934 Moody St', 'address2': '', '...",+17813731548,(781) 373-1548,13861.742449
484,IGbKO5xKMZrTsPtmDYaLwQ,chateau-restaurant-waltham,Chateau Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/Nc-Xoi...,False,https://www.yelp.com/biz/chateau-restaurant-wa...,343,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.0,"{'latitude': 42.3783249, 'longitude': -71.239395}","[pickup, delivery]",$$,"{'address1': '195 School St', 'address2': None...",+17818943339,(781) 894-3339,14224.750388
